# Visualizing Genetic Circuits: Creating Diagrams with SBOL Visual

In this notebook, we will load the SBOL file created in the previous notebook and visualize the transcriptional unit using two different libraries: `dnaplotlib` and `parasbolv`.


In [ ]:
# Setup
# Install the necessary libraries
# pip install sbol3 excel2sbol dnaplotlib parasbolv

import sbol3
import sbol_utilities.component as sbcomp
import dnaplotlib as dpl
import parasbolv as psv

import matplotlib.pyplot as plt
from matplotlib import gridspec

gs = gridspec.GridSpec(1, 1)

# Load the SBOL file
rdf_file = "transcriptional_unit.rdf"


In [ ]:
# Load the SBOL document from the RDF file
doc = sbol3.Document()
doc.read(rdf_file)

### Retrieving Ordered Components from a Transcriptional Unit

This function, `get_ordered_components`, extracts the components in the correct sequential order from a transcriptional unit based on the constraints defined in the SBOL3 document. It does so by identifying the `SubComponent` instances within the transcriptional unit and resolving them to their corresponding `Component` objects using the `instance_of` property. The function ensures the correct order of components by traversing the "meets" relationships, which specify how one component follows another in the sequence.


In [ ]:
def get_ordered_components(doc, transcriptional_unit_id):

    # Retrieve the transcriptional unit component
    transcriptional_unit = doc.find(transcriptional_unit_id)
    
    # Create a dictionary to hold the order of components
    order_map = {}
    
    # Populate the order_map based on the constraints
    for constraint in transcriptional_unit.constraints:
        if constraint.restriction == sbol3.SBOL_MEETS:  # Check if the constraint is a 'meets' relationship
            subject = constraint.subject.lookup()
            obj = constraint.object.lookup()
            order_map[subject.display_id] = obj.display_id
    
    # Use the order_map to determine the order of components
    ordered_components = []
    
    # Find the first component (one that is not in the object of any constraint)
    first_subcomponent = None
    for feature in transcriptional_unit.features:
        if isinstance(feature, sbol3.SubComponent) and feature.display_id not in order_map.values():
            first_subcomponent = feature
            break
    
    # Traverse the map to get the ordered list of components
    current_subcomponent = first_subcomponent
    while current_subcomponent:
        # Get the associated component from the instance_of property
        component = current_subcomponent.instance_of.lookup()
        ordered_components.append(component)
        next_subcomponent_id = order_map.get(current_subcomponent.display_id)
        current_subcomponent = transcriptional_unit.find(next_subcomponent_id) if next_subcomponent_id else None
    
    return ordered_components

In [ ]:
ordered_components = get_ordered_components(doc, 'https://github.com/SynBioDex/SBOL-Notebooks/TranscriptionalUnit1')

Now that we have an ordered list of DNA parts, we will visualise these components using 2 SBOL Visual Libraries

In [ ]:

def get_plot_from_design(design, filename):
    fig = plt.figure(figsize=(1,0.3))
    ax_dna = plt.subplot(gs[0])

    # Create the DNAplotlib renderer
    dr = dpl.DNARenderer()
    dr.linewidth = 0.8

    start, end = dr.renderDNA(ax_dna, design, dr.SBOL_part_renderers())
    ax_dna.set_xlim([start, end])
    ax_dna.set_ylim([-10,30.0])
    ax_dna.set_aspect('equal')
    ax_dna.axis('off')

    fig.savefig(f'{filename}.png', dpi=300, transparent=True)
    plt.close('all')

In [ ]:
# We will first write a simple helper function that will convert our SBOL Part into SBOL Visual Parts using DNAPlotlib

# SBOL Type to DNAPlotlib Type Map:
dpl_type_map = {
    sbol3.SO_PROMOTER: 'Promoter',
    sbol3.SO_RBS: 'RBS',
    sbol3.SO_CDS: 'CDS',
    sbol3.SO_TERMINATOR: 'Terminator'
}

def get_dnaplotlib_part(component: sbol3.Component):
    return {
        'type': dpl_type_map[component.roles[0]],
        'fwd': True
    }

design = [get_dnaplotlib_part(x) for x in ordered_components]

get_plot_from_design(design, 'tu')

### Adding Custom Properties and Styling to DNAplotlib Visual Elements

In this section, we define a property map, `dpl_property_map`, that specifies custom styling options for each type of genetic part (e.g., promoters, RBS, CDS, terminators). These properties include line width, color, and other visual attributes, allowing us to customize the appearance of each part when rendering with DNAplotlib.

We then define a helper function, `get_dnaplotlib_part_props`, which maps each SBOL component to its corresponding DNAplotlib properties based on its role. This function returns a dictionary with the part type, orientation, and styling options.

Using this function, we generate a list of part properties for the ordered components and render the transcriptional unit with DNAplotlib, applying the specified styles. The final figure is saved as a PNG file with the specified custom styling.


In [ ]:

dpl_property_map = {
    sbol3.SO_PROMOTER: {'linewidth':0.8, 'color':'red', 'edge_color':(0.00, 0.00, 0.00), 'y_extent':9, 'x_extent':13, 'arrowhead_height':2, 'arrowhead_length':3},
    sbol3.SO_RBS: {'linewidth':0.3, 'color':'black', 'edge_color':(0.00, 0.00, 0.00), 'x_extent':7, 'start_pad':1},
    sbol3.SO_CDS: {'linewidth':0.2, 'color':'green', 'edge_color':(0.00, 0.00, 0.00), 'x_extent':14, 'y_extent':2.2,'arrowhead_height':2,'arrowhead_length':4},
    sbol3.SO_TERMINATOR: {'linewidth':0.5, 'color':'black', 'edge_color':(0.00, 0.00, 0.00), 'x_extent':6, 'y_extent':6}
}

def get_dnaplotlib_part_props(component: sbol3.Component):
    return {
        'type': dpl_type_map[component.roles[0]],
        'fwd': True,
        'opts': dpl_property_map[component.roles[0]]
    }

design = [get_dnaplotlib_part_props(x) for x in ordered_components]
get_plot_from_design(design, 'tu_color')
